In [ ]:
! pip install openpyxl

In [1]:
import finpy_tse as fpy
import pandas as pd
import sqlite3 as db

In [2]:
portfollio = 'افران توان اوج پالایش وخارزم اعتلا وبوعلی تمشک گوهران وهامون ومدیر وتوسم وسبحان بوعلی وسکاب مدیریت وصنعت کیان وسپه ومهان'
portfollio_eng = 'vmahan vsepah kian vsanaat modiriat vsakab booali vsobhan vtosam vmodir vhamoon goharan tameshk vbooali alaa vkharazm palayesh owj tavaan afran'
my_pf = portfollio.split()
my_pf_eng = portfollio_eng.split()
my_pf_eng = my_pf_eng[::-1]

اتصال به دیتابیس بورس

### جایگزین کردن پرتفوی جدید با پرتفوی قدیمی

In [ ]:
df_portfollio = pd.read_excel('portfollio.xlsx')
conn = db.connect('bours.db')
cur = conn.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS portfollio(
    symbol TEXT PRIMARY KEY,
    property INT,
    average_purchase INT,
    head_to_head INT,
    last_price INT,
    final_price INT,
    cost INT,
    current_value INT,
    current_profit_percentage INT,
    current_profit INT,
    past_profit INT,
    total_profit INT
    )
""")
df_portfollio.to_sql('portfollio',conn,if_exists='replace',index=False)
conn.commit()
cur.close()
conn.close()

ابتدا عمق بازار دریافتی باید مناسب سازی جهت اضافه کردن با دیتابیس شود

### ساخت جدول یا تیبل

In [ ]:
conn = db.connect('my_bours.db')
cur = conn.cursor()
table_name = 'price_daily'

# cur.execute(f"""CREATE TABLE IF NOT EXISTS {table_name}(
   Date TEXT,
   Time TEXT,
   Open INT,
   High INT,
   Low INT,
   Close INT,
   Final INT,
   Volume INT,
   Value INT,
   No INT,
   Ticker TEXT PRIMARY KEY,
   Name TEXT,
   Market TEXT
   )
""")
cur.close()
conn.close()

ساخت تیبل نیازی نبود
### دریافت قیمت روزانه کل پرتفوی خودم و افزودن به دیتا بیس

In [ ]:


conn = db.connect('my_bours2.db')
cur = conn.cursor()


for sym in my_pf:
    df = fpy.Get_Price_History(
                    stock=sym ,
                    start_date='1402-03-21',
                    end_date='1402-03-22',
                    ignore_date=False,
                    adjust_price=False,
                    show_weekday=False,
                    double_date=False)

    df.reset_index(inplace=True)
    df = df.rename(columns={"J-Date": "Date"})

    #df.to_excel('ttt.xlsx',index=False)

    df.to_sql('price_daily',conn,if_exists='append',index=False)
    print(f'{sym} is added to db')


"""
 df.to_sql(table_name,conn,if_exists='replace',index=False)
 
 # insert the dataframe into an existing table in the database file
 
 df.to_sql(tale_name, conn, if_exists='append', index=False)
"""

# close the connection object
conn.commit()
cur.close()
conn.close()

## دستورات پانداس برای نرمال کردن داده ها

In [ ]:
df['length'] = df['length']/df['length'].max() # method 1 ) simple normalizing

df['length'] = (df['length']-df['length'].min())/(df['lengh'].max()-df['lengh'].min()) # method 2 ) min-max method

df['length'] = (df['length']-df['length'].mean())/df['length'].std()

### کويری گرفتن قیمت سهم در تاریخ دلخواه

In [ ]:

conn = db.connect('my_bours2.db')
df_query = pd.read_sql_query("SELECT  Ticker,Close, Date FROM price_daily WHERE Date BETWEEN '1402-01-14' AND '1402-01-20'", conn)

conn.close()
df_query

In [ ]:
df_chart = df_query.sort_values('Date')

In [ ]:
df_chart

## دریافت ریز معاملات

In [73]:
i = 0
print(my_pf_eng[i])

afran


In [68]:
symbol = my_pf[i]
eng_symbol = my_pf_eng[i]
df_price_details =fpy.Get_IntradayTrades_History(
                    stock=symbol,
                    start_date='1402-04-13',
                    end_date='1402-04-14',
                    jalali_date=True,
                    combined_datatime=False,
                    show_progress=True)
df_price_details.head()

Progress :  100.0 %


Volume  Price
J-Date     Time                   
1402-04-13 09:26:47     467  17389
           09:29:22   10149  17420
           09:29:22    1500  17420
           09:29:22    9148  17440
           09:29:22    5159  17443

اضافه کردن ریز معاملات به دیتا بیس

In [69]:
conn = db.connect('my_bours_price_details.db')
cur = conn.cursor()

df_price_details.reset_index(inplace=True)
df_price_details_new = df_price_details.rename(columns={"J-Date": "Date"})    
df_price_details_new.to_sql(eng_symbol,conn,if_exists='append',index=False)

conn.commit()
cur.close()
conn.close()

## دریافت عمق فقط برای چند نماد

In [ ]:
startDate = '1402-01-28'
endDate = '1402-01-30'

conn = db.connect('my_bours_omgh.db')
cur = conn.cursor()


for sym,eng in zip(my_pf,my_pf_eng):
    df_omgh = fpy.Get_IntradayOB_History(
            stock=sym,
            start_date= startDate,
            end_date= endDate,
            jalali_date=True,
            combined_datatime=False,
            show_progress=True)
    
    df_omgh.reset_index(inplace=True)
    df_omgh_new = df_omgh.rename(columns={"J-Date": "Date"})    
    df_omgh_new.to_sql(f'{eng}_omgh_daily',conn,if_exists='append',index=False)
    print(f'{sym} is done')


# close the connection object
conn.commit()
cur.close()
conn.close()

## دریافت عمق فقط برای یک نماد

In [125]:
i = 2
print(my_pf_eng[i])
symbol = my_pf[i]
eng = my_pf_eng[i]

owj


In [123]:
startDate = '1402-04-10'
endDate = '1402-04-14'
stk = symbol

conn = db.connect('my_bours_omgh.db')
cur = conn.cursor()
df_omgh = fpy.Get_IntradayOB_History(
            stock= stk ,
            start_date= startDate,
            end_date= endDate,
            jalali_date=True,
            combined_datatime=False,
            show_progress=True)
print('done')

df_omgh.reset_index(inplace=True)
df_omgh = df_omgh.rename(columns={"J-Date": "Date"})
df_omgh.to_sql(f'{eng}_omgh_daily',conn,if_exists='append',index=False)

conn.commit()
cur.close()
conn.close()

Progress :  100.0 %
done


##  دریافت سابقه ارزش صف در زمان بسته‌شدن بازار

In [118]:
df_que = fpy.Get_Queue_History(
            stock='توان',
            start_date='1402-03-01',
            end_date='1402-03-20',
            show_per_capita=True,
            show_weekday=False,
            double_date=False,
            show_progress=True)
df_que

Progress :  100.0 %


,Day_UL,Day_LL,Value,Time,BQ_Value,SQ_Value,BQPC,SQPC
J-Date,,,,,,,,
1402-03-01,18729,22889,1051683869514,12:30:00,0,0,0,0
1402-03-02,19061,23295,957971538612,12:30:00,0,0,0,0
1402-03-03,19007,23229,807288854395,12:29:59,0,0,0,0
1402-03-06,19300,23588,539209625597,12:29:00,0,0,0,0
1402-03-07,18614,22750,964254421040,12:30:00,0,0,0,0
1402-03-08,17756,21700,633582247731,12:29:59,0,0,0,0
1402-03-09,17993,21991,785654385549,12:30:00,0,0,0,0
1402-03-10,17957,21947,776824363587,12:29:59,0,0,0,0
1402-03-13,17682,21610,224338225132,12:29:50,0,0,0,0


##  دریافت سابقه ارزش صف در زمان بسته‌شدن بازار

In [ ]:
df_CWI = fpy.Get_CWI_History(
            start_date='1402-01-05',
            end_date='1402-03-21',
            ignore_date=False,
            just_adj_close=False,
            show_weekday=True,
            double_date=False)
df_CWI

## دریافت سابقه شاخص کل هم‌وزن

In [ ]:
df_EWI = fpy.Get_EWI_History(
            start_date='1402-01-05',
            end_date='1402-03-21',
            ignore_date=False,
            just_adj_close=True,
            show_weekday=False,
            double_date=False)
df_EWI

## دریافت سابقه شاخص 50 شرکت فعال‌تر

In [ ]:
df_ACT50 = fpy.Get_ACT50_History(
            start_date='1402-01-05',
            end_date='1402-03-21',
            ignore_date=False,
            just_adj_close=False,
            show_weekday=False,
            double_date=False)
df_ACT50

## دریافت سابقه شاخص 30 شرکت بزرگ

In [ ]:
df_LCI30 = fpy.Get_LCI30_History(
            start_date='1402-01-05',
            end_date='1402-03-21',
            ignore_date=False,
            just_adj_close=False,
            show_weekday=False,
            double_date=False)
df_LCI30